In [4]:
import pandas as pd
import numpy as np
import cv2
import dlib
import os

In [15]:
face_cascade = cv2.CascadeClassifier('../face_recognition/haarcascade_frontalface_default.xml')
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [5]:
IMG_PATH="../facial_expressions/"

In [6]:
emotions=["anger", "disgust", "fear", "happy", "neutral", "sad"]

In [11]:
face_dim=128

In [12]:
import matplotlib.pyplot as plt

In [45]:
def ret_keypoints(impath):
    try:
        image = cv2.imread(impath)
    except:
        return None
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.resize(gray, (face_dim, face_dim))
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    if len(faces)==0:
        param=[0,0,face_dim,face_dim]
    else:
        param=faces[0]
    shape=predictor(gray,dlib.rectangle(*param))
    xlist=[]
    ylist=[]
    for i in range(68):
        xlist.append(float(shape.part(i).x))
        ylist.append(float(shape.part(i).y))
    xmean = np.mean(xlist)
    ymean = np.mean(ylist)
#     plt.imshow(gray,cmap='gray')
#     plt.scatter(xlist,ylist, marker='.')
#     plt.show()
    xcentral = [(x-xmean) for x in xlist]
    ycentral = [(y-ymean) for y in ylist]
    res=[]
    for (x,y) in zip(xcentral,ycentral):
        res.append(x)
        res.append(y)
    return (np.asarray(res)/face_dim+1)/2

In [50]:
def prep_keypoints():
    X_inp=[]
    y_inp=[]
    for em in emotions:
        ltt=os.listdir(IMG_PATH+em)
        lnltt=len(ltt)
        for idx,imn in enumerate(ltt):
            print("\r",idx+1,'/',lnltt,end=" ")
            landmarks=ret_keypoints(IMG_PATH+em+"/"+imn)
            if landmarks is not None:
                X_inp.append(landmarks)
                yy=np.zeros(len(emotions))
                yy[emotions.index(em)]=1
                y_inp.append(yy)
        print()
    return np.asarray(X_inp),np.asarray(y_inp)

In [51]:
XXD,YYD=prep_keypoints()

 124 / 124 
 84 / 84 
 67 / 67 
 127 / 127 6 / 127 
 123 / 123 
 132 / 132 


In [68]:
s=np.arange(len(XXD))
for i in range(5):
    np.random.shuffle(s)
    XXD=XXD[s]
    YYD=YYD[s]

In [71]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

In [72]:
model=Sequential()
model.add(Dense(512,activation='relu', input_shape=(68*2,)))
# model.add(BatchNormalization())
# model.add(Dropout(0.5))
# model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(256,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Dense(128,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(len(emotions),activation='softmax'))

In [73]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               70144     
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024      
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               3

In [74]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [75]:
cut=80
XDB=XXD[cut:]
YDB=YYD[cut:]

XTB=XXD[:cut]
YTB=YYD[:cut]

In [81]:
model.fit(XDB,YDB,batch_size=16,epochs=200,validation_data=(XTB,YTB))

Train on 577 samples, validate on 80 samples
Epoch 1/500
577/577 [==============================] - 0s 691us/sample - loss: 1.2996 - accuracy: 0.4905 - val_loss: 2.1716 - val_accuracy: 0.2375
Epoch 2/500
577/577 [==============================] - 0s 346us/sample - loss: 1.3917 - accuracy: 0.4645 - val_loss: 3.1831 - val_accuracy: 0.3000
Epoch 3/500
577/577 [==============================] - 0s 338us/sample - loss: 1.4039 - accuracy: 0.4731 - val_loss: 1.6290 - val_accuracy: 0.4375
Epoch 4/500
577/577 [==============================] - 0s 340us/sample - loss: 1.3921 - accuracy: 0.4333 - val_loss: 1.6439 - val_accuracy: 0.3375
Epoch 5/500
577/577 [==============================] - 0s 349us/sample - loss: 1.3849 - accuracy: 0.4610 - val_loss: 2.3363 - val_accuracy: 0.1750
Epoch 6/500
577/577 [==============================] - 0s 348us/sample - loss: 1.3353 - accuracy: 0.4593 - val_loss: 1.6244 - val_accuracy: 0.3375
Epoch 7/500
577/577 [==============================] - 0s 347us/sample - 

577/577 [==============================] - 0s 385us/sample - loss: 1.2686 - accuracy: 0.4939 - val_loss: 1.5664 - val_accuracy: 0.3000
Epoch 57/500
577/577 [==============================] - 0s 325us/sample - loss: 1.2602 - accuracy: 0.5130 - val_loss: 1.5442 - val_accuracy: 0.3875
Epoch 58/500
577/577 [==============================] - 0s 298us/sample - loss: 1.2669 - accuracy: 0.5182 - val_loss: 1.5171 - val_accuracy: 0.4250
Epoch 59/500
577/577 [==============================] - 0s 338us/sample - loss: 1.2588 - accuracy: 0.5147 - val_loss: 1.8063 - val_accuracy: 0.3375
Epoch 60/500
577/577 [==============================] - 0s 404us/sample - loss: 1.2379 - accuracy: 0.5165 - val_loss: 3.0640 - val_accuracy: 0.2875
Epoch 61/500
577/577 [==============================] - 0s 440us/sample - loss: 1.2675 - accuracy: 0.5043 - val_loss: 2.4258 - val_accuracy: 0.3250
Epoch 62/500
577/577 [==============================] - 0s 331us/sample - loss: 1.2730 - accuracy: 0.5095 - val_loss: 1.8001 

577/577 [==============================] - 0s 333us/sample - loss: 1.2490 - accuracy: 0.5286 - val_loss: 1.7873 - val_accuracy: 0.3375
Epoch 112/500
577/577 [==============================] - 0s 334us/sample - loss: 1.2230 - accuracy: 0.5234 - val_loss: 1.6833 - val_accuracy: 0.3250
Epoch 113/500
577/577 [==============================] - 0s 327us/sample - loss: 1.1949 - accuracy: 0.5407 - val_loss: 1.5848 - val_accuracy: 0.3875
Epoch 114/500
577/577 [==============================] - 0s 300us/sample - loss: 1.1597 - accuracy: 0.5477 - val_loss: 1.8529 - val_accuracy: 0.3375
Epoch 115/500
577/577 [==============================] - 0s 361us/sample - loss: 1.1912 - accuracy: 0.5199 - val_loss: 1.4704 - val_accuracy: 0.3875
Epoch 116/500
577/577 [==============================] - 0s 358us/sample - loss: 1.2267 - accuracy: 0.5061 - val_loss: 1.9434 - val_accuracy: 0.3000
Epoch 117/500
577/577 [==============================] - 0s 333us/sample - loss: 1.1536 - accuracy: 0.5546 - val_loss: 1

577/577 [==============================] - 0s 455us/sample - loss: 1.1988 - accuracy: 0.5615 - val_loss: 1.7651 - val_accuracy: 0.3750
Epoch 167/500
577/577 [==============================] - 0s 356us/sample - loss: 1.1638 - accuracy: 0.5598 - val_loss: 1.8874 - val_accuracy: 0.3250
Epoch 168/500
577/577 [==============================] - 0s 324us/sample - loss: 1.1626 - accuracy: 0.5355 - val_loss: 1.6163 - val_accuracy: 0.3750
Epoch 169/500
577/577 [==============================] - 0s 345us/sample - loss: 1.1344 - accuracy: 0.5477 - val_loss: 1.6724 - val_accuracy: 0.3375
Epoch 170/500
577/577 [==============================] - 0s 366us/sample - loss: 1.1473 - accuracy: 0.5407 - val_loss: 1.6496 - val_accuracy: 0.4125
Epoch 171/500
577/577 [==============================] - 0s 379us/sample - loss: 1.1352 - accuracy: 0.5806 - val_loss: 1.7163 - val_accuracy: 0.3750
Epoch 172/500
577/577 [==============================] - 0s 355us/sample - loss: 1.1065 - accuracy: 0.5529 - val_loss: 2

577/577 [==============================] - 0s 316us/sample - loss: 1.1000 - accuracy: 0.5615 - val_loss: 1.6629 - val_accuracy: 0.4000
Epoch 222/500
577/577 [==============================] - 0s 357us/sample - loss: 1.0368 - accuracy: 0.5875 - val_loss: 2.0789 - val_accuracy: 0.3250
Epoch 223/500
577/577 [==============================] - 0s 303us/sample - loss: 1.1199 - accuracy: 0.5598 - val_loss: 1.6287 - val_accuracy: 0.4500
Epoch 224/500
577/577 [==============================] - 0s 332us/sample - loss: 1.0851 - accuracy: 0.5806 - val_loss: 1.6919 - val_accuracy: 0.4125
Epoch 225/500
577/577 [==============================] - 0s 305us/sample - loss: 1.1065 - accuracy: 0.5442 - val_loss: 2.1153 - val_accuracy: 0.3625
Epoch 226/500
577/577 [==============================] - 0s 336us/sample - loss: 1.1198 - accuracy: 0.5477 - val_loss: 1.7393 - val_accuracy: 0.4000
Epoch 227/500
577/577 [==============================] - 0s 379us/sample - loss: 1.1015 - accuracy: 0.5477 - val_loss: 2

577/577 [==============================] - 0s 645us/sample - loss: 1.0346 - accuracy: 0.5893 - val_loss: 1.7764 - val_accuracy: 0.3500
Epoch 277/500
577/577 [==============================] - 0s 299us/sample - loss: 1.0223 - accuracy: 0.5875 - val_loss: 1.7577 - val_accuracy: 0.4250
Epoch 278/500
577/577 [==============================] - 0s 391us/sample - loss: 1.0487 - accuracy: 0.6049 - val_loss: 1.7949 - val_accuracy: 0.4000
Epoch 279/500
577/577 [==============================] - 0s 571us/sample - loss: 1.0822 - accuracy: 0.5719 - val_loss: 1.6788 - val_accuracy: 0.3500
Epoch 280/500
577/577 [==============================] - 0s 584us/sample - loss: 1.0887 - accuracy: 0.5875 - val_loss: 2.8988 - val_accuracy: 0.3000
Epoch 281/500
577/577 [==============================] - 0s 513us/sample - loss: 1.0543 - accuracy: 0.5893 - val_loss: 1.8441 - val_accuracy: 0.4125
Epoch 282/500
577/577 [==============================] - 0s 414us/sample - loss: 1.1299 - accuracy: 0.5754 - val_loss: 1

577/577 [==============================] - 0s 410us/sample - loss: 1.0435 - accuracy: 0.5875 - val_loss: 2.1468 - val_accuracy: 0.3625
Epoch 332/500
577/577 [==============================] - 0s 300us/sample - loss: 1.0855 - accuracy: 0.5875 - val_loss: 1.8257 - val_accuracy: 0.4250
Epoch 333/500
577/577 [==============================] - 0s 366us/sample - loss: 1.0327 - accuracy: 0.5979 - val_loss: 2.0406 - val_accuracy: 0.3250
Epoch 334/500
577/577 [==============================] - 0s 369us/sample - loss: 1.0481 - accuracy: 0.5927 - val_loss: 1.7879 - val_accuracy: 0.3875
Epoch 335/500
577/577 [==============================] - 0s 373us/sample - loss: 1.0036 - accuracy: 0.6170 - val_loss: 1.9022 - val_accuracy: 0.3875
Epoch 336/500
577/577 [==============================] - 0s 632us/sample - loss: 1.0611 - accuracy: 0.5789 - val_loss: 1.9487 - val_accuracy: 0.3750
Epoch 337/500
577/577 [==============================] - 0s 399us/sample - loss: 1.0638 - accuracy: 0.5823 - val_loss: 1

577/577 [==============================] - 0s 442us/sample - loss: 0.9759 - accuracy: 0.6343 - val_loss: 2.4858 - val_accuracy: 0.3000
Epoch 387/500
577/577 [==============================] - 0s 645us/sample - loss: 0.9926 - accuracy: 0.6326 - val_loss: 2.1022 - val_accuracy: 0.3125
Epoch 388/500
577/577 [==============================] - 0s 336us/sample - loss: 1.0327 - accuracy: 0.6014 - val_loss: 2.6343 - val_accuracy: 0.3500
Epoch 389/500
577/577 [==============================] - 0s 449us/sample - loss: 0.9647 - accuracy: 0.6187 - val_loss: 1.9728 - val_accuracy: 0.3750
Epoch 390/500
577/577 [==============================] - 0s 392us/sample - loss: 1.0397 - accuracy: 0.5910 - val_loss: 2.2009 - val_accuracy: 0.3250
Epoch 391/500
577/577 [==============================] - 0s 295us/sample - loss: 1.0037 - accuracy: 0.6118 - val_loss: 1.9881 - val_accuracy: 0.3625
Epoch 392/500
577/577 [==============================] - 0s 309us/sample - loss: 1.0214 - accuracy: 0.5893 - val_loss: 2

577/577 [==============================] - 0s 327us/sample - loss: 1.0040 - accuracy: 0.5893 - val_loss: 2.1416 - val_accuracy: 0.3625
Epoch 442/500
577/577 [==============================] - 0s 340us/sample - loss: 0.9485 - accuracy: 0.6205 - val_loss: 1.7577 - val_accuracy: 0.4625
Epoch 443/500
577/577 [==============================] - 0s 354us/sample - loss: 0.9860 - accuracy: 0.6205 - val_loss: 1.9583 - val_accuracy: 0.3125
Epoch 444/500
577/577 [==============================] - 0s 317us/sample - loss: 0.9538 - accuracy: 0.6291 - val_loss: 2.3254 - val_accuracy: 0.3750
Epoch 445/500
577/577 [==============================] - 0s 330us/sample - loss: 0.9317 - accuracy: 0.6447 - val_loss: 2.9273 - val_accuracy: 0.3375
Epoch 446/500
577/577 [==============================] - 0s 293us/sample - loss: 0.9494 - accuracy: 0.6308 - val_loss: 2.5297 - val_accuracy: 0.2625
Epoch 447/500
577/577 [==============================] - 0s 309us/sample - loss: 0.9790 - accuracy: 0.6395 - val_loss: 2

577/577 [==============================] - 0s 300us/sample - loss: 0.9476 - accuracy: 0.6360 - val_loss: 2.1466 - val_accuracy: 0.3250
Epoch 497/500
577/577 [==============================] - 0s 302us/sample - loss: 1.0235 - accuracy: 0.5979 - val_loss: 2.3225 - val_accuracy: 0.3125
Epoch 498/500
577/577 [==============================] - 0s 321us/sample - loss: 0.9465 - accuracy: 0.6256 - val_loss: 2.5792 - val_accuracy: 0.3000
Epoch 499/500
577/577 [==============================] - 0s 374us/sample - loss: 0.9256 - accuracy: 0.6430 - val_loss: 2.5938 - val_accuracy: 0.3500
Epoch 500/500
577/577 [==============================] - 0s 490us/sample - loss: 0.9890 - accuracy: 0.6205 - val_loss: 2.9113 - val_accuracy: 0.3625


In [78]:
(model.predict(XTB).argmax(axis=1)==YTB.argmax(axis=1)).sum()/len(XTB)

0.325

In [79]:
model.predict(XTB).argmax(axis=1)

array([5, 3, 3, 5, 3, 0, 0, 3, 3, 3, 3, 1, 5, 5, 3, 3, 3, 0, 3, 3, 0, 5,
       3, 1, 3, 3, 5, 5, 3, 5, 0, 0, 0, 5, 5, 3, 5, 3, 5, 3, 0, 5, 3, 3,
       0, 0, 3, 0, 3, 5, 5, 3, 0, 3, 3, 0, 0, 1, 5, 3, 0, 0, 3, 3, 5, 3,
       3, 3, 0, 5, 3, 0, 5, 3, 0, 3, 5, 3, 5, 3])

In [80]:
YTB.argmax(axis=1)

array([5, 0, 3, 0, 3, 0, 1, 2, 1, 3, 4, 0, 0, 5, 3, 4, 4, 3, 1, 0, 1, 5,
       3, 0, 5, 1, 0, 2, 3, 2, 1, 4, 4, 5, 5, 0, 5, 3, 5, 1, 4, 2, 3, 0,
       0, 0, 4, 1, 1, 0, 1, 4, 1, 5, 3, 3, 1, 0, 0, 3, 0, 4, 3, 5, 0, 3,
       1, 4, 4, 0, 5, 1, 2, 2, 0, 4, 5, 3, 0, 2])